# Comparação

Pense Bayes, Segunda Edição

Copyright 2020 Allen B. Downey

Licença: [Attribution-NonCommercial-ShareAlike 4.0 International (CC BY-NC-SA 4.0)](https://creativecommons.org/licenses/by-nc-sa/4.0/)

In [1]:
# If we're running on Colab, install empiricaldist
# https://pypi.org/project/empiricaldist/

import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    !pip install empiricaldist

In [2]:
# Get utils.py

from os.path import basename, exists

def download(url):
    filename = basename(url)
    if not exists(filename):
        from urllib.request import urlretrieve
        local, _ = urlretrieve(url, filename)
        print('Downloaded ' + local)
    
download('https://github.com/AllenDowney/ThinkBayes2/raw/master/soln/utils.py')

In [3]:
from utils import set_pyplot_params
set_pyplot_params()

Este capítulo introduz distribuições conjuntas, que são uma ferramenta essencial para trabalhar com distribuições de mais do que uma variável.

Utilizá-los-emos para resolver um problema idiota no nosso caminho para resolver um problema real.

O problema tolo é descobrir quão altas são duas pessoas, dado que apenas uma é mais alta do que a outra.

O verdadeiro problema é avaliar os jogadores de xadrez (ou participantes noutros tipos de competição) com base no resultado de um jogo.

Para construir distribuições conjuntas e calcular as probabilidades para estes problemas, utilizaremos produtos externos e operações similares.  E é por aí que vamos começar.

## Operações externas

Muitas operações úteis podem ser expressas como o "produto exterior" de duas sequências, ou outro tipo de operação "exterior".

Suponha que tem sequências como `x` e `y`:

In [4]:
x = [1, 3, 5]
y = [2, 4]

O produto exterior destas sequências é uma matriz que contém o produto de cada par de valores, um de cada sequência.

Existem várias formas de calcular produtos exteriores, mas a que eu penso ser a mais versátil é uma "grelha de malha".

NumPy fornece uma função chamada `meshgrid` que calcula uma grelha de malha.  Se lhe dermos duas sequências, ele devolve duas arrays.

In [5]:
import numpy as np

X, Y = np.meshgrid(x, y)

A primeira matriz contém cópias de `x` dispostas em filas, onde o número de filas é o comprimento de `y`.

In [6]:
X

A segunda matriz contém cópias de `y` dispostas em colunas, onde o número de colunas é o comprimento de `x`.

In [7]:
Y

Como as duas matrizes têm o mesmo tamanho, podemos usá-las como operandos para funções aritméticas como a multiplicação.

In [8]:
X * Y

Este é o resultado é o produto exterior de `x` e `y`.

Podemos ver isso mais claramente se o colocarmos num "DataFrame":

In [9]:
import pandas as pd

df = pd.DataFrame(X * Y, columns=x, index=y)
df

Os valores de `x` aparecem como nomes de coluna; os valores de `y` aparecem como etiquetas de linha.

Cada elemento é o produto de um valor de `x` e de um valor de `y`.

Podemos utilizar redes de malha para calcular outras operações, como a soma exterior, que é uma matriz que contém o *sum* de elementos de `x` e elementos de `y`.

In [10]:
X + Y

Podemos também utilizar operadores de comparação para comparar elementos de `x` com elementos de `y`.

In [11]:
X > Y

O resultado é um conjunto de valores booleanos.

Pode ainda não ser óbvio porque é que estas operações são úteis, mas veremos exemplos em breve.

Com isso, estamos prontos para enfrentar um novo problema Bayesiano.

## How Tall Is A?

Suponha que eu escolho duas pessoas da população de homens adultos nos EUA; chamar-lhes-ei A e B. Se virmos que A é mais alto do que B, qual é a altura de A?

Para responder a esta pergunta:

1. Vou utilizar informação de fundo sobre a altura dos homens nos EUA para formar uma distribuição prévia da altura,

2. Construirei uma distribuição prévia conjunta de altura para A e B (e explicarei o que isso é),

3. Depois actualizarei o anterior com a informação de que A é mais alto, e 

4. Da distribuição posterior conjunta vou extrair a distribuição posterior de altura para A.

Nos EUA, a altura média dos adultos do sexo masculino é de 178 cm e o desvio padrão é de 7,7 cm.  A distribuição não é exactamente normal, porque nada no mundo real o é, mas a distribuição normal é um modelo bastante bom da distribuição real, pelo que podemos usá-la como distribuição prévia para A e B.

Aqui está um conjunto de valores igualmente espaçados de 3 desvios padrão abaixo da média para 3 desvios padrão acima (arredondados um pouco para cima).

In [12]:
mean = 178
qs = np.arange(mean-24, mean+24, 0.5)

SciPy fornece uma função chamada `norm` que representa uma distribuição normal com uma dada média e desvio padrão, e fornece `pdf`, que avalia a função de densidade de probabilidade (PDF) da distribuição normal:

In [13]:
from scipy.stats import norm

std = 7.7
ps = norm(mean, std).pdf(qs)

As densidades de probabilidade não são probabilidades, mas se as colocarmos num `Pmf` e as normalizarmos, o resultado é uma aproximação discreta da distribuição normal.

In [14]:
from empiricaldist import Pmf

prior = Pmf(ps, qs)
prior.normalize()

Aqui está o que parece.

In [15]:
from utils import decorate

prior.plot(style='--', color='C5')

decorate(xlabel='Height in cm',
         ylabel='PDF',
         title='Approximate distribution of height for men in U.S.')

Esta distribuição representa aquilo em que acreditamos sobre as alturas de "A" e "B" antes de levarmos em conta os dados que "A" é mais alto.

## Distribuição conjunta

O passo seguinte é construir uma distribuição que represente a probabilidade de cada par de alturas, a que se chama uma distribuição conjunta. Os elementos da distribuição conjunta são

$$P(A_x~\mathrm{and}~B_y)$$$

que é a probabilidade de `A` ter $x$ cm de altura e `B` ter $y$ cm de altura, para todos os valores de $x$ e $y$.

Neste momento, tudo o que sabemos sobre `A` e `B` é que são homens residentes nos EUA, pelo que as suas alturas são independentes; ou seja, saber a altura de `A` não fornece qualquer informação adicional sobre a altura de `B`.

Nesse caso, podemos calcular as probabilidades conjuntas desta forma:

$$P(A_x~\mathrm{and}~B_y) = P(A_x)~P(B_y)$$

Cada probabilidade conjunta é o produto de um elemento da distribuição de `x` e de um elemento da distribuição de `y`.

Assim, se tivermos objectos "Pmf" que representam a distribuição de altura para "A" e "B", podemos calcular a distribuição conjunta através do cálculo do produto externo das probabilidades em cada "Pmf".

A seguinte função pega em dois objectos `Pmf` e devolve um `DataFrame` que representa a distribuição conjunta.

In [16]:
def make_joint(pmf1, pmf2):
    """Compute the outer product of two Pmfs."""
    X, Y = np.meshgrid(pmf1, pmf2)
    return pd.DataFrame(X * Y, columns=pmf1.qs, index=pmf2.qs)

Os nomes das colunas no resultado são as quantidades de `pmf1`; as etiquetas das linhas são as quantidades de `pmf2`.

Neste exemplo, as distribuições anteriores para `A` e `B` são as mesmas, pelo que podemos calcular a distribuição anterior conjunta desta forma:

In [17]:
joint = make_joint(prior, prior)
joint.shape

O resultado é um 'DataFrame' com possíveis alturas de 'A' ao longo das colunas, alturas de 'B' ao longo das filas, e as probabilidades conjuntas como elementos.

Se o prior é normalizado, o prior conjunto também é normalizado.

In [18]:
joint.to_numpy().sum()

Para somar todos os elementos, convertemos o "DataFrame" para um "NumPy array" antes de chamar "soma".  Caso contrário, o `DataFrame.sum' calcularia as somas das colunas e retornaria uma `Série'.

In [19]:
series = joint.sum()
series.shape

## Visualizando a Distribuição Conjunta

A seguinte função utiliza `pcolormesh` para traçar a distribuição conjunta.

In [20]:
import matplotlib.pyplot as plt

def plot_joint(joint, cmap='Blues'):
    """Plot a joint distribution with a color mesh."""
    vmax = joint.to_numpy().max() * 1.1
    plt.pcolormesh(joint.columns, joint.index, joint, 
                   cmap=cmap,
                   vmax=vmax,
                   shading='nearest')
    plt.colorbar()
    
    decorate(xlabel='A height in cm',
             ylabel='B height in cm')

Eis como é a distribuição prévia conjunta.

In [21]:
plot_joint(joint)
decorate(title='Joint prior distribution of height for A and B')

Como seria de esperar, a probabilidade é mais elevada (mais escura) perto da média e desce mais longe da média.

Outra forma de visualizar a distribuição conjunta é um gráfico de contorno.

In [22]:
def plot_contour(joint):
    """Plot a joint distribution with a contour."""
    plt.contour(joint.columns, joint.index, joint,
                linewidths=2)
    decorate(xlabel='A height in cm',
             ylabel='B height in cm')

In [23]:
plot_contour(joint)
decorate(title='Joint prior distribution of height for A and B')

Cada linha representa um nível de probabilidade igual.  

## Probabilidade

Agora que temos uma distribuição prévia conjunta, podemos actualizá-la com os dados, que é que `A` é mais alto que `B`.

Cada elemento da distribuição conjunta representa uma hipótese sobre as alturas de `A` e `B`.

Para calcular a probabilidade de cada par de quantidades, podemos extrair os nomes das colunas e os rótulos das linhas do anterior, desta forma:

In [24]:
x = joint.columns
y = joint.index

E utilizá-los para calcular uma grelha de malha.

In [25]:
X, Y = np.meshgrid(x, y)

O `X' contém cópias das quantidades em `x', que são alturas possíveis para `A'.   Y` contém cópias das quantidades em `y`, que são possíveis alturas para `B`.

Se compararmos `X` e `Y`, o resultado é um conjunto booleano:

In [26]:
A_taller = (X > Y)
A_taller.dtype

Para calcular as probabilidades, utilizarei `np.where` para fazer um array com `1` onde `A_taller` é `True` e 0 noutro lugar.

In [27]:
a = np.where(A_taller, 1, 0)

Para visualizar este conjunto de probabilidades, vou colocar um "DataFrame" com os valores de "x" como nomes de colunas e os valores de "y" como etiquetas de linha.

In [28]:
likelihood = pd.DataFrame(a, index=x, columns=y)

Aqui está o que parece:

In [29]:
plot_joint(likelihood, cmap='Oranges')
decorate(title='Likelihood of A>B')

A probabilidade dos dados é 1 onde `X > Y` e 0 noutro lugar.

## A Actualização

Temos um prévio, temos uma probabilidade, e estamos prontos para a actualização.  Como é habitual, o posterior não normalizado é o produto do anterior e a probabilidade.

In [30]:
posterior = joint * likelihood

Vou utilizar a seguinte função para normalizar o posterior:

In [31]:
def normalize(joint):
    """Normalize a joint distribution."""
    prob_data = joint.to_numpy().sum()
    joint /= prob_data
    return prob_data

In [32]:
normalize(posterior)

E aqui está o que parece.

In [33]:
plot_joint(posterior)
decorate(title='Joint posterior distribution of height for A and B')

Todos os pares em que "B" é mais alto do que "A" foram eliminados.  O resto dos pares posteriores parece o mesmo que o anterior, excepto que foi renormalizado.

## Distribuições Marginais

A distribuição posterior conjunta representa aquilo em que acreditamos sobre as alturas de "A" e "B", dadas as distribuições anteriores e a informação de que "A" é mais alta.

A partir desta distribuição conjunta, podemos calcular as distribuições posteriores para `A` e `B`.  Para ver como, vamos começar com um problema mais simples.

Suponhamos que queremos saber a probabilidade de `A` ter 180 cm de altura.  Podemos seleccionar a coluna a partir da distribuição conjunta onde `x=180`.

In [34]:
column = posterior[180]
column.head()

Esta coluna contém probabilidades posteriores para todos os casos em que `x=180`; se as somarmos, obtemos a probabilidade total de `A` ter 180 cm de altura.

In [35]:
column.sum()

É cerca de 3%.

Agora, para obter a distribuição posterior de altura para 'A', podemos somar todas as colunas, desta forma:

In [36]:
column_sums = posterior.sum(axis=0)
column_sums.head()

O argumento `eixo=0` significa que queremos somar as colunas.

O resultado é uma "Série" que contém todas as alturas possíveis para "A" e a sua probabilidade.  Por outras palavras, é a distribuição das alturas para "A".

Podemos colocá-lo num `Pmf` como este:

In [37]:
marginal_A = Pmf(column_sums)

Quando extraímos a distribuição de uma única variável de uma distribuição conjunta, o resultado chama-se uma **distribuição marginal***.

O nome vem de uma visualização comum que mostra a distribuição conjunta no meio e as distribuições marginais nas margens.

Eis como é a distribuição marginal para `A`:

In [38]:
marginal_A.plot(label='Posterior for A')

decorate(xlabel='Height in cm',
         ylabel='PDF',
         title='Posterior distribution for A')

Da mesma forma, podemos obter a distribuição posterior da altura para `B` somando as filas e colocando o resultado num `Pmf`.

In [39]:
row_sums = posterior.sum(axis=1)
marginal_B = Pmf(row_sums)

Aqui está o que parece.

In [40]:
marginal_B.plot(label='Posterior for B', color='C1')

decorate(xlabel='Height in cm',
         ylabel='PDF',
         title='Posterior distribution for B')

Vamos colocar o código desta secção numa função:

In [41]:
def marginal(joint, axis):
    """Compute a marginal distribution."""
    return Pmf(joint.sum(axis=axis))

O `marginal` toma como parâmetros uma distribuição conjunta e um número de eixo:

* Se `eixo=0`, devolve o marginal da primeira variável (o do eixo x);

* Se `eixo=1`, devolve a marginal da segunda variável (a do eixo y).


Assim, podemos calcular os dois marginais desta forma:

In [42]:
marginal_A = marginal(posterior, axis=0)
marginal_B = marginal(posterior, axis=1)

Eis o que eles parecem, juntamente com o anterior.

In [43]:
prior.plot(style='--', label='Prior', color='C5')
marginal_A.plot(label='Posterior for A')
marginal_B.plot(label='Posterior for B')

decorate(xlabel='Height in cm',
         ylabel='PDF',
         title='Prior and posterior distributions for A and B')

Como seria de esperar, a distribuição posterior para `A` é deslocada para a direita e a distribuição posterior para `B` é deslocada para a esquerda.

Podemos resumir os resultados através do cálculo dos meios posteriores:

In [44]:
prior.mean()

In [45]:
print(marginal_A.mean(), marginal_B.mean())

Com base na observação de que `A` é mais alto que `B`, estamos inclinados a acreditar que `A` é um pouco mais alto que a média, e que `B` é um pouco mais baixo.

Note-se que as distribuições posteriores são um pouco mais estreitas do que as anteriores.  Podemos quantificar isso através do cálculo dos seus desvios padrão.

In [46]:
prior.std()

In [47]:
print(marginal_A.std(), marginal_B.std())

Os desvios padrão das distribuições posteriores são um pouco menores, o que significa que estamos mais certos sobre as alturas de `A` e `B` depois de as compararmos.

## Posteriors condicionais

Agora suponha-se que medimos `A` e descobrimos que ele tem 170 cm de altura.  O que é que isso nos diz sobre `B`?

Na distribuição conjunta, cada coluna corresponde a uma altura possível para `A`.  Podemos seleccionar a coluna que corresponde à altura de 170 cm desta forma:

In [48]:
column_170 = posterior[170]

O resultado é uma "Série" que representa possíveis alturas para "B" e as suas probabilidades relativas.

Estas probabilidades não são normalizadas, mas podemos normalizá-las desta forma:

In [49]:
cond_B = Pmf(column_170)
cond_B.normalize()

Fazer um `Pmf` copia os dados por defeito, para que possamos normalizar `cond_B` sem afectar a `coluna_170` ou o `posterior`.

O resultado é a distribuição condicional de altura para `B` dado que `A` tem 170 cm de altura.

Aqui está o que parece:

In [50]:
prior.plot(style='--', label='Prior', color='C5')
marginal_B.plot(label='Posterior for B', color='C1')
cond_B.plot(label='Conditional posterior for B', 
            color='C4')

decorate(xlabel='Height in cm',
         ylabel='PDF',
         title='Prior, posterior and conditional distribution for B')

A distribuição posterior condicional é cortada a 170 cm, porque estabelecemos que `B` é mais curto que `A`, e `A` é 170 cm.



## Dependência e Independência

Quando construímos a distribuição prévia conjunta, eu disse que as alturas de `A` e `B` eram independentes, o que significa que saber uma delas não fornece qualquer informação sobre a outra.

Por outras palavras, a probabilidade condicional $P(A_x | B_y)$ é a mesma que a probabilidade incondicional $P(A_x)$.

Mas na distribuição posterior, $A$ e $B$ não são independentes.

Se soubermos que `A` é mais alto que `B`, e soubermos quão alto `A` é, isso dá-nos informação sobre `B`.

A distribuição condicional que acabámos de calcular demonstra esta dependência.

## Resumo

Neste capítulo, começámos com as operações "exteriores", como produto exterior, que utilizávamos para construir uma distribuição conjunta.

Em geral, não se pode construir uma distribuição conjunta a partir de duas distribuições marginais, mas no caso especial em que as distribuições são independentes, pode.

Estendemos o processo de actualização Bayesian e aplicámo-lo a uma distribuição conjunta.  Depois, da distribuição conjunta posterior extraímos distribuições posteriores marginais e distribuições posteriores condicionais.



## Exercícios

**Exercício:** Com base nos resultados do exemplo anterior, calcular a distribuição condicional posterior para `A` dado que `B` é de 180 cm.

Dica: Utilize `loc` para seleccionar uma fila de um `DataFrame`.

In [51]:
# Solution goes here

In [52]:
# Solution goes here

**Exercício:** Suponha que estabelecemos que `A` é mais alto que `B`, mas não sabemos qual é a altura de `B`.

Agora escolhemos uma mulher aleatória, `C`, e descobrimos que ela é mais curta do que `A` em pelo menos 15 cm.  Calcular as distribuições posteriores para as alturas de `A` e `C`.

A altura média das mulheres nos EUA é de 163 cm; o desvio padrão é de 7,3 cm.

In [53]:
# Solution goes here

In [54]:
# Solution goes here

In [55]:
# Solution goes here

In [56]:
# Solution goes here

In [57]:
# Solution goes here

In [58]:
# Solution goes here

In [59]:
# Solution goes here

In [60]:
# Solution goes here

**Exercício:** [The Elo rating system](https://en.wikipedia.org/wiki/Elo_rating_system) é uma forma de quantificar o nível de habilidade dos jogadores para jogos como o xadrez.

Baseia-se num modelo da relação entre as classificações dos jogadores e o resultado de um jogo.  Especificamente, se $R_A$ é a classificação do jogador `A` e $R_B$ é a classificação do jogador `B`, a probabilidade de `A` bater `B` é dada pelo [logistic function](https://en.wikipedia.org/wiki/Logistic_function):

$$P(\mathrm{A~beats~B}) = \frac{1}{1 + 10^{(R_B-R_A)/400}}$$$

Os parâmetros 10 e 400 são escolhas arbitrárias que determinam o alcance das classificações.  No xadrez, o intervalo é de 100 a 2800.

Note-se que a probabilidade de ganhar depende apenas da diferença nas classificações.  Como exemplo, se $R_A$ exceder $R_B$ em 100 pontos, a probabilidade de `A` ganhar é

In [61]:
1 / (1 + 10**(-100/400))

Suponhamos que `A` tem uma classificação actual de 1600, mas não temos a certeza de que seja exacta.  Poderíamos descrever a sua verdadeira classificação com uma distribuição normal com média 1600 e desvio padrão 100, para indicar a nossa incerteza.

E suponha que `B` tem uma classificação actual de 1800, com o mesmo nível de incerteza.

Depois `A` e `B` jogam e `A` ganha.  Como devemos actualizar as suas classificações?

Para responder a esta pergunta:

1. Construir distribuições prévias para `A` e `B`.

2. Utilizá-los para construir uma distribuição conjunta, assumindo que as distribuições anteriores são independentes.

3. Utilizar a função logística acima para calcular a probabilidade do resultado sob cada hipótese conjunta.  

4. Utilizar a articulação anterior e a probabilidade de calcular a articulação posterior. 

5. Extrair e traçar os cartazes marginais para `A` e `B`.

6. Calcular os meios posteriores para `A` e `B`.  Quanto é que as suas classificações devem mudar com base neste resultado?

In [62]:
# Solution goes here

In [63]:
# Solution goes here

In [64]:
# Solution goes here

In [65]:
# Solution goes here

In [66]:
# Solution goes here

In [67]:
# Solution goes here

In [68]:
# Solution goes here

In [69]:
# Solution goes here

In [70]:
# Solution goes here

In [71]:
# Solution goes here

In [72]:
# Solution goes here

In [73]:
# Solution goes here